In [1]:
import zipfile
import os
import shutil
import json
import pandas as pd

from collections import Counter
from glob import glob

In [2]:
file_dir = "../"
ex_dir = "../test_data"

In [3]:
#zip 파일 압축 풀기
for name in os.listdir(file_dir):
    if ".zip" in name:
        zipfile.ZipFile(file_dir+name).extractall(ex_dir)
        print("완료: "+name)

완료: 220121testdata-20220121T061729Z-001.zip


In [4]:
#zip 파일 삭제
for name in os.listdir(file_dir):
    if ".zip" in name:
        os.remove(file_dir+name)
        print("삭제: "+name)

삭제: 220121testdata-20220121T061729Z-001.zip


In [5]:
data_path = "../test_data/220121testdata/01.원천데이터/식의약용 자생식물 분석 데이터/"
label_path = "../test_data/220121testdata/02.라벨링데이터/식의약용 자생식물 분석 데이터/"
move_path = "../yolov5/data/plant_test_img/"

In [6]:
#move path로 json 파일 모두 이동
for one in os.listdir(label_path):
    one_ = os.listdir(label_path+one)
    for two in one_:
        try:
            two_ = os.listdir(label_path+one+"/"+two)
            for f in two_:
                shutil.move(label_path+one+"/"+two+"/"+f, move_path)
        except:
            shutil.move(label_path+one+"/"+two, move_path)

In [7]:
#move path로 이미지 파일 모두 이동
for one in os.listdir(data_path):
    one_ = os.listdir(data_path+one)
    for two in one_:
        try:
            two_ = os.listdir(data_path+one+"/"+two)
            for f in two_:
                shutil.move(data_path+one+"/"+two+"/"+f, move_path)
        except:
            shutil.move(data_path+one+"/"+two, move_path)

In [8]:
json_list = glob(move_path+"*.json")
img_list = glob(move_path+"*.jpg")

In [9]:
#json과 이미지 개수 확인
print("JSON 개수:", len(json_list))
print("IMG 개수:", len(img_list))

JSON 개수: 540
IMG 개수: 540


In [10]:
plants = []

for jf in json_list:
    with open(jf, "r", encoding="UTF-8") as f:
        json_data = json.load(f)
        json.dumps(json_data, indent="\t", ensure_ascii=False)
        plants.append(json_data["annotations"]["object_class_mlsft_nm"]+"("+
                      json_data["annotations"]["object_class_sclas_nm"]+")")

In [11]:
#test 이미지 분포 확인
dict(Counter(plants))

{'까마귀쪽나무(열매)': 30,
 '좁은잎천선과(열매)': 30,
 '참식나무(잎-앞면)': 30,
 '메밀(꽃)': 30,
 '메밀(잎-뒷면)': 30,
 '구실잣밤나무(열매)': 30,
 '돈나무(열매)': 30,
 '큰조롱(꽃)': 30,
 '순비기나무(꽃)': 30,
 '순비기나무(열매)': 30,
 '참꽃나무(꽃)': 30,
 '참가시나무(열매)': 30,
 '황근(꽃)': 30,
 '황근(열매)': 30,
 '해국(꽃)': 30,
 '한라꽃향유(꽃)': 30,
 '백량금(열매)': 30,
 '꽝꽝나무(열매)': 30}

In [12]:
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return(x,y,w,h)

In [13]:
class_dict={
    '순비기나무(꽃)':"0",
    '메밀(꽃)':"1",
    '큰조롱(꽃)':"2",
    '순비기나무(열매)':"3",
    '해국(꽃)':"4",
    '참꽃나무(꽃)':"5",
    '참가시나무(열매)':"6",
    '백량금(열매)':"7",
    '참식나무(잎-앞면)':"8",
    '좁은잎천선과(열매)':"9",
    '구실잣밤나무(열매)':"10",
    '돈나무(열매)':"11",
    '황근(열매)':"12",
    '한라꽃향유(꽃)':"13",
    '까마귀쪽나무(열매)':"14",
    '꽝꽝나무(열매)':"15",
    '메밀(잎-뒷면)':"16",
    '황근(꽃)':"17"
}

In [14]:
def json2txt(json_path, txt_path):
    with open(json_path, "r", encoding="UTF-8") as f:
        json_data = json.load(f)
        json.dumps(json_data, indent="\t", ensure_ascii=False)
        
    rsoltn = json_data["images"]["rsoltn"].split(", ")
    size = (int(rsoltn[0]), int(rsoltn[1]))
    
    xcrdnt = json_data["annotations"]["pyn_xcrdnt"].split(",")
    ycrdnt = json_data["annotations"]["pyn_ycrdnt"].split(",")
    
    x_loc = list(map(lambda x: int(x), xcrdnt))
    y_loc = list(map(lambda x: int(x), ycrdnt))
    
    xmin = min(x_loc)
    xmax = max(x_loc)
    ymin = min(y_loc)
    ymax = max(y_loc)
    
    box = (xmin, xmax, ymin, ymax)
    conv_box = convert(size, box)
    
    class_num = class_dict[json_data["annotations"]["object_class_mlsft_nm"]+"("+json_data["annotations"]["object_class_sclas_nm"]+")"]
    
    row = class_num+" "+str(conv_box[0])+" "+str(conv_box[1])+" "+str(conv_box[2])+" "+str(conv_box[3])
        
    with open(txt_path, "w") as f:
        f.write("\n".join([row])+"\n")

In [15]:
file_list = list(map(lambda x:x.split(".json")[0], json_list))

In [16]:
#yolo format에 맞는 txt 파일 생성
for f in file_list: json2txt(f+".json", f+".txt")

In [17]:
test = list(map(lambda x: x.split("yolov5/")[1]+".jpg", file_list))

In [18]:
with open("../yolov5/data/plant_test.txt", "w") as f:
    f.write("\n".join(test)+"\n")

In [19]:
#경로 설정
os.chdir("../yolov5")

#경로 확인
os.getcwd()

'C:\\Users\\user\\plant\\yolov5'

In [20]:
!python val.py --weights plant_256.pt --data data/plant_test.yaml --imgsz 256 --device 0

requirements: opencv-python>=4.1.2 not found and is required by YOLOv5, attempting auto-update...
requirements: 'pip install opencv-python>=4.1.2' skipped (offline)


val: data=data/plant_test.yaml, weights=['plant_256.pt'], batch_size=32, imgsz=256, conf_thres=0.001, iou_thres=0.6, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  v6.0-189-gaf00134 torch 1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
Model Summary: 213 layers, 7058671 parameters, 0 gradients

val: Scanning 'data\plant_test' images and labels...:   0%|          | 0/540 [00:00<?, ?it/s]
val: Scanning 'data\plant_test' images and labels...1 found, 0 missing, 0 empty, 0 corrupted:   0%|          | 1/540 [00:07<1:09:46,  7.77s/it]
val: Scanning 'data\plant_test' images and labels...5 found, 0 missing, 0 empty, 0 corrupted:   1%|          | 5/540 [00:07<10:31,  1.18s/it]  
val: Scanning 'data\plant_test' images and labels...8 found, 0 missing, 0 empty, 0 corrupted:   1%|1         | 8/540 [00:

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  24%|##3       | 4/17 [00:05<00:12,  1.01it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  29%|##9       | 5/17 [00:05<00:08,  1.45it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  35%|###5      | 6/17 [00:06<00:05,  1.94it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  41%|####1     | 7/17 [00:06<00:04,  2.37it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  47%|####7     | 8/17 [00:06<00:03,  2.78it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  53%|#####2    | 9/17 [00:06<00:02,  3.28it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  59%|#####8    | 10/17 [00:06<00:01,  3.78it/s]
               Class     Images     Labels          P 